# LSTM (장기 의존성 학습 모델)
- RNN 구조에서 발생하는 문제를 해결하기 위해 구조를 변경한 모델 
- 기존의 RNN은 학습의 횟수가 증가하면 과거의 기억이 잃는다. 
    - 반복 학습을 하면서 가중치의 변화량이 0에 가까워짐을 의미 
    - 손실 함수 -> tanh -> 반복적으로 tanh() 함수를 실행하게 되면 변화량 0
- LSTM은 반복 학습을 하면서 과거의 기억 중 장기 기억과, 단기 기억으로 나눠주는 역할
    - 반복 학습을 하면서 tanh() 중복적으로 실행이 되면 기울기 변화량 줄어듬.
    - 장기 기억은 tank() 함수를 실행하지 않고 -> 가중치의 변화를 계속 주겠다.
    - 셀이라는 공간에 장기 기억 정보를 전달 
    - 게이트로 정도를 선택적으로 보관, 삭제, 출력

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler

In [2]:
# 데이터로드 
df = pd.read_csv("../csv/AAPL.csv")

In [ ]:
# Date 컬럼의 dtype을 시계열 데이터로 변환 
# 문자형 데이터를 시계열 데이터로 변환 -> 
# 그래프 시각화할때 X축의 데이터로 사용하기 위함
df['Date'] = pd.to_datetime(
    df['Date']
)

In [4]:
# 특정 컬럼 선택 
df = df[['Date', 'Adj Close', 'Volume']]
# 결측치가 포함되어있는 행을 모두 제거 
df.dropna(axis=0, inplace=True)

In [5]:
# 독립 데이터, 종속 데이터를 생성 
X_all = df[['Adj Close', 'Volume']]
Y_all = df[['Adj Close']]

In [6]:
# 데이터 8:2 비율로 나눠준다. 
split_idx = int(len(df) * 0.8)
X_train, X_test = X_all[ : split_idx], X_all[split_idx : ]
Y_train, Y_test = Y_all[ : split_idx], Y_all[split_idx : ]
print(X_train.shape, X_test.shape)
print(Y_train.shape, Y_test.shape)

(7770, 2) (1943, 2)
(7770, 1) (1943, 1)


In [54]:
# 초기 설정 값
# 시계열 데이터의 구간(과거 데이터의 개수)
window = 60
# 학습 루프에서 루프의 수 
epochs = 20
# 학습 데이터의 배치의 개수
x_batch = 64
# optimizer lr 값 (학습율)
lr = 0.001
# 은닉 층의 뉴런의 개수 
hidden_cnt = 64
# 랜덤의 일반화
torch.manual_seed(42)
# 은닉층, 셀 사용 여부
head_type = 'h'

In [11]:
class WindowDataset(Dataset):
    # 입력 데이터와 타깃 데이터를 받아서 
    # 구간의 수만큼 입력데이터를 잘라주고 
    # 해당 구간 바로 다음 행의 타깃 데이터를 생성
    # 구간의 데이터와 다음행의 타깃 데이터를 되돌려준다
    # DataLoader 클래스에서 사용하기 위함
    def __init__(
            self, _x, _y, _window
    ):
        self.x = _x
        self.y = _y
        self.window = _window
        # n의 값은 구간의 데이터 시작 지점의 최대값
        self.n = len(_x) - _window
    
    def __len__(self):
        # window 의 수치가 x의 길이보다 큰 경우에는 1을 되돌려준다. 
        return max(self.n, 1)
    
    def __getitem__(self, idx):
        # idx : 구간의 시점 지점 -> 최대 값은 self.n
        # idx부터 idx+window-1 지점까지의 데이터 -> (window, 2)
        x_window = self.x[idx : idx + self.window]
        # idx+window 위치의 데이터  -> 단일값 생성
        y_next = self.y[idx + self.window]
        # tensor형의 변환 
        x_tensor = torch.from_numpy(x_window)
        y_tensor = torch.from_numpy(y_next)

        return x_tensor, y_tensor

In [12]:
# 데이터들 스케일링 -> MinMaxScaler
x_scaler = MinMaxScaler().fit(X_train)
y_scaler = MinMaxScaler().fit(Y_train)

X_train = x_scaler.transform(X_train)
X_test = x_scaler.transform(X_test)
Y_train = y_scaler.transform(Y_train)
Y_test = y_scaler.transform(Y_test)

In [61]:
# WindowDataset 생성 
# 객체을 생성하고 getitem() 함수를 사용한다면 window(60) 구간의 데이터와
# 해당 구간 다음 행의 종가 데이터를 되돌려주는 class
train_ds = WindowDataset(X_train, Y_train, window)
test_ds = WindowDataset(X_test, Y_test, window)

In [62]:
# Dataloader -> WindowDataset  class를 이용하여 
# ds_data에서 len() 함수를 이용하여 해당 구간에서 가장 마지막에 사용 가능한 시작 지점

train_dl = DataLoader(train_ds, shuffle=True, drop_last= True, 
                      batch_size=x_batch)
test_dl = DataLoader(test_ds, shuffle=False, drop_last = False, 
                     batch_size=256)

In [63]:
# i = 0
# for x, y in test_dl:
#     print("구간 데이터 : ",x)
#     # print("다음 날의 종가 :", y)
#     if i == 2 : 
#         break
#     i += 1

In [64]:
# LSTM 모델 정의 

class LSTMReg(nn.Module):
    # h : 마지막의 은닉층의 데이터를 이용 (RNN -> 기억 소실 , LSTM -> 장기 적인 기억은 유지)
    # c : 셀의 데이터를 이용 
    # h_c : 은닉층 + 셀의 데이터를 이용
    def __init__(
            self, 
            input_size, hidden_size, 
            num_layers = 1, dropout = 0.0, 
            bidirectional = False, head_type = 'h'
    ):
        super().__init__()

        # lstm 기본 설정 
        self.lstm = nn.LSTM(
            input_size = input_size, 
            hidden_size= hidden_size, 
            num_layers= num_layers, 
            dropout= dropout, 
            bidirectional= bidirectional
        )

        # bidirectional이 True인 경우 차원의 개수 * 2
        out_dim = hidden_size * (2 if bidirectional else 1)
        # head_type은 은닉층만 사용하거나 셀만 사용 -> 차원의 개수 * 1 
        if head_type in ['h', 'c']:
            head_in = out_dim
        elif head_type == 'h_c':
            head_in  = out_dim * 2
        else:
            print('head_type Error')

        self.head = nn.Sequential(
            nn.Linear(head_in, 1)
        )
        self.head_type = head_type
    
    # 순전파 함수 
    def forward(self, x):
        # 순전파의 예측 결과 값
        out, (h_n, c_n) = self.lstm(x)
        # 은닉층 중 마지막 은닉층을 저장 
        h_last = h_n[-1]
        # 셀 마지막 셀 
        c_last = torch.tanh(c_n[-1])

        if self.head_type == 'h':
            feat = h_last
        elif self.head_type == 'c':
            feat = c_last
        elif self.head_type == 'h_c':
            feat = torch.cat( [h_last, c_last], dim = -1 )
        
        return self.head(feat)

In [65]:
model = LSTMReg(input_size=2, hidden_size=hidden_cnt, 
                head_type=head_type)
model

LSTMReg(
  (lstm): LSTM(2, 64)
  (head): Sequential(
    (0): Linear(in_features=64, out_features=1, bias=True)
  )
)